In [9]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

## La función

In [ ]:
def resumen_tabla(df, nombre_tabla, top_nulls=10):
    print("="*80)
    print(f"RESUMEN DATA UNDERSTANDING - {nombre_tabla}")
    print("="*80)

    # Dimensiones
    print("\nDimensiones (filas, columnas):")
    print(df.shape)

    # Tipos de datos
    print("\nTipos de datos:")
    print(df.dtypes.value_counts())

    # Columnas categóricas y numéricas
    cat_cols = df.select_dtypes(include="object").columns
    num_cols = df.select_dtypes(include=["int64", "float64"]).columns

    print("\nNúmero de columnas categóricas:", len(cat_cols))
    print("Número de columnas numéricas:", len(num_cols))

    # Valores nulos
    nulls = df.isnull().sum()
    null_percent = (nulls / len(df)) * 100

    nulls_df = pd.DataFrame({
        "Nulos": nulls,
        "% Nulos": null_percent
    }).sort_values("% Nulos", ascending=False)

    print(f"\nTop {top_nulls} columnas con más valores nulos:")
    display(nulls_df.head(top_nulls))

    # Columnas con alto porcentaje de nulos
    high_nulls = nulls_df[nulls_df["% Nulos"] > 40]
    print("\nColumnas con más de 40% de valores nulos:")
    display(high_nulls)

    # Columnas constantes
    constant_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
    print("\nColumnas constantes o con un solo valor:")
    print(constant_cols)

    # Duplicados
    duplicates = df.duplicated().sum()
    print("\nFilas duplicadas:", duplicates)

    # Estadísticas básicas (solo numéricas)
    if len(num_cols) > 0:
        print("\nEstadísticas básicas (primeras 10 columnas numéricas):")
        display(df[num_cols].describe().T.head(10))

    print("\nFin del resumen de la tabla.")

## Rutas

In [ ]:
DATA_PATH = "../datos_examen"

application = pd.read_parquet(os.path.join(DATA_PATH, "application_train.parquet"))
bureau = pd.read_parquet(os.path.join(DATA_PATH, "bureau.parquet"))
bureau_balance = pd.read_parquet(os.path.join(DATA_PATH, "bureau_balance.parquet"))
previous_application = pd.read_parquet(os.path.join(DATA_PATH, "previous_application.parquet"))
pos_cash = pd.read_parquet(os.path.join(DATA_PATH, "POS_CASH_balance.parquet"))
installments = pd.read_parquet(os.path.join(DATA_PATH, "installments_payments.parquet"))
credit_card = pd.read_parquet(os.path.join(DATA_PATH, "credit_card_balance.parquet"))
columns_desc = pd.read_parquet(os.path.join(DATA_PATH, "HomeCredit_columns_description.parquet"))

## Descripción de application

In [ ]:
resumen_tabla(application, "application_train")

### Tabla: application_train

**Descripción:**  
La tabla `application_train` contiene la información asociada a la solicitud de crédito actual de cada cliente al momento de la postulación. Es la tabla central del proyecto y la única que incorpora la variable objetivo `TARGET`, por lo que constituye la base sobre la cual se entrenan y evalúan los modelos de predicción de incumplimiento de pago.

**Llave principal:**  
`SK_ID_CURR`

**Granularidad:**  
Cada fila representa una solicitud de crédito individual asociada a un cliente único.

**Variables relevantes:**  
`TARGET`, `AMT_INCOME_TOTAL`, `AMT_CREDIT`, `AMT_ANNUITY`, `AMT_GOODS_PRICE`, `DAYS_BIRTH`, `DAYS_EMPLOYED`, `CNT_CHILDREN`, `CODE_GENDER`, `NAME_CONTRACT_TYPE`

**Observaciones:**  
El dataset presenta alta dimensionalidad, con un total de 122 columnas, de las cuales 106 son numéricas y 16 categóricas. La variable objetivo `TARGET` se encuentra fuertemente desbalanceada, con aproximadamente un 8% de registros correspondientes a clientes en situación de incumplimiento, lo que representa un desafío relevante para el modelado.  
Se identifican múltiples columnas con un porcentaje elevado de valores nulos, principalmente variables relacionadas con características físicas de la vivienda (áreas, número de pisos, antigüedad del edificio y tipo de construcción), superando en muchos casos el 60% de valores faltantes. Estas variables deberán ser evaluadas en la fase de preparación para determinar su imputación o eliminación.  
No se detectan columnas constantes ni filas duplicadas. Asimismo, se observa la presencia de valores anómalos conocidos, como el valor `365243` en la variable `DAYS_EMPLOYED`, que corresponde a una codificación inválida del tiempo de empleo y deberá ser tratada en etapas posteriores.


## Descripción bureau

In [ ]:
resumen_tabla(bureau, "bureau")

### Tabla: bureau

**Descripción:**  
La tabla `bureau` contiene información histórica de los créditos que los clientes han mantenido en otras instituciones financieras, reportados al buró de crédito. Esta tabla permite capturar el comportamiento crediticio externo del cliente, complementando la información interna disponible en la solicitud actual.

**Llave principal:**  
`SK_ID_BUREAU`

**Granularidad:**  
Cada fila representa un crédito individual reportado en el buró, asociado a un cliente a través de la clave `SK_ID_CURR`.

**Variables relevantes:**  
`SK_ID_CURR`, `DAYS_CREDIT`, `CREDIT_DAY_OVERDUE`, `AMT_CREDIT_SUM`, `AMT_CREDIT_SUM_DEBT`, `AMT_CREDIT_MAX_OVERDUE`, `CNT_CREDIT_PROLONG`, `DAYS_CREDIT_ENDDATE`, `DAYS_ENDDATE_FACT`

**Observaciones:**  
La tabla presenta una relación uno-a-muchos con la tabla principal `application_train`, ya que un mismo cliente puede tener múltiples créditos externos registrados. Cuenta con un total de 1.716.428 registros y 17 columnas, predominando las variables numéricas.  
Se identifican columnas con un porcentaje muy elevado de valores nulos, particularmente `AMT_ANNUITY` (más del 71%) y `AMT_CREDIT_MAX_OVERDUE` (más del 65%), lo que indica información incompleta sobre condiciones financieras de ciertos créditos y requerirá una evaluación cuidadosa en la fase de preparación.  
Existen también valores faltantes relevantes en variables temporales como `DAYS_ENDDATE_FACT` y `DAYS_CREDIT_ENDDATE`, lo que refleja que no todos los créditos cuentan con fechas de cierre o término registradas.  
Desde el punto de vista de los rangos, se observan valores extremos en montos crediticios y saldos de deuda, incluyendo montos muy elevados y valores negativos en `AMT_CREDIT_SUM_DEBT`, lo que sugiere la presencia de ajustes, cancelaciones o correcciones contables. No se detectan columnas constantes ni registros duplicados en la tabla.


## Descripción de bureau_balance

In [ ]:
resumen_tabla(bureau_balance, "bureau_balance")

### Tabla: bureau_balance

**Descripción:**  
La tabla `bureau_balance` contiene el historial mensual del estado de los créditos externos reportados en el buró de crédito. Proporciona información temporal sobre el comportamiento de cada crédito a lo largo del tiempo, indicando su estado mes a mes hasta el momento actual o su cierre.

**Llave principal:**  
`SK_ID_BUREAU`

**Granularidad:**  
Cada fila representa el estado de un crédito específico (`SK_ID_BUREAU`) en un mes determinado, indicado por la variable `MONTHS_BALANCE`.

**Variables relevantes:**  
`SK_ID_BUREAU`, `MONTHS_BALANCE`, `STATUS`

**Observaciones:**  
La tabla presenta un volumen extremadamente alto de registros, con aproximadamente 27,3 millones de filas y solo 3 columnas, lo que confirma su naturaleza de serie temporal mensual. No se identifican valores nulos, columnas constantes ni registros duplicados, lo que indica una buena consistencia estructural.  
La variable `MONTHS_BALANCE` representa meses relativos al momento actual, con valores negativos que indican meses en el pasado y un máximo de 0 correspondiente al mes más reciente. La variable categórica `STATUS` codifica el estado del crédito en cada mes, siendo fundamental para capturar atrasos, pagos regulares y situaciones de incumplimiento a lo largo del tiempo.  
Debido a su granularidad mensual y a la relación uno-a-muchos con la tabla `bureau`, esta tabla no puede integrarse directamente al nivel cliente y requiere agregaciones temporales previas para su uso en el modelado.


## Descripción de previous_application

In [ ]:
resumen_tabla(previous_application, "previous_application")

### Tabla: previous_application

**Descripción:**  
La tabla `previous_application` contiene información detallada sobre las solicitudes de crédito anteriores realizadas por los clientes dentro de Home Credit. Permite analizar el historial interno de postulaciones del cliente, incluyendo montos solicitados, montos aprobados, condiciones financieras y decisiones asociadas a cada solicitud previa.

**Llave principal:**  
`SK_ID_PREV`

**Granularidad:**  
Cada fila representa una solicitud de crédito previa asociada a un cliente, vinculada a la tabla principal mediante la clave `SK_ID_CURR`.

**Variables relevantes:**  
`SK_ID_CURR`, `SK_ID_PREV`, `AMT_APPLICATION`, `AMT_CREDIT`, `AMT_ANNUITY`, `AMT_GOODS_PRICE`, `HOUR_APPR_PROCESS_START`, `NAME_CONTRACT_STATUS`, `RATE_DOWN_PAYMENT`, `AMT_DOWN_PAYMENT`

**Observaciones:**  
La tabla presenta un volumen elevado de registros, con más de 1,6 millones de filas y 37 columnas, y una combinación equilibrada de variables numéricas y categóricas. Se observa una alta proporción de valores nulos en varias columnas, destacando especialmente `RATE_INTEREST_PRIVILEGED` y `RATE_INTEREST_PRIMARY`, que superan el 99% de valores faltantes, lo que limita significativamente su utilidad analítica.  
También se identifican columnas con más del 40% de valores nulos relacionadas con pagos iniciales y con fechas clave del ciclo de vida del crédito (`AMT_DOWN_PAYMENT`, `RATE_DOWN_PAYMENT`, `DAYS_FIRST_DUE`, `DAYS_LAST_DUE`, entre otras), lo que sugiere que dicha información no está disponible para todas las solicitudes.  
No se detectan columnas constantes ni registros duplicados. Desde el punto de vista de los valores, se observan montos crediticios con rangos amplios y algunos valores atípicos, incluyendo valores negativos o cercanos a cero en variables de pago inicial, lo que deberá ser evaluado y tratado en la fase de preparación.  
Debido a su relación uno-a-muchos con la tabla `application_train`, esta tabla requiere agregaciones previas a nivel cliente antes de su integración al dataset final.


## Descripción de pos_cash

In [ ]:
resumen_tabla(pos_cash, "pos_cash")

### Tabla: POS_CASH_balance

**Descripción:**  
La tabla `POS_CASH_balance` contiene información mensual sobre créditos de consumo (POS y cash loans) asociados a solicitudes de crédito previas dentro de Home Credit. Permite analizar el comportamiento de pago del cliente a lo largo del tiempo para este tipo de productos financieros.

**Llave principal:**  
`SK_ID_PREV`

**Granularidad:**  
Cada fila representa el estado de un crédito de consumo específico en un mes determinado, identificado por la variable `MONTHS_BALANCE`.

**Variables relevantes:**  
`SK_ID_CURR`, `SK_ID_PREV`, `MONTHS_BALANCE`, `CNT_INSTALMENT`, `CNT_INSTALMENT_FUTURE`, `SK_DPD`, `SK_DPD_DEF`, `NAME_CONTRACT_STATUS`

**Observaciones:**  
La tabla presenta un volumen elevado de registros, con más de 10 millones de filas y 8 columnas, lo que confirma su naturaleza de serie temporal mensual. La mayoría de las variables corresponden a indicadores numéricos relacionados con el número de cuotas y los días de atraso.  
Los valores nulos son escasos y se concentran únicamente en las variables `CNT_INSTALMENT` y `CNT_INSTALMENT_FUTURE`, con un porcentaje inferior al 0,3%, lo que indica una buena calidad general de los datos. No se detectan columnas constantes ni registros duplicados.  
Se observan valores extremos en las variables `SK_DPD` y `SK_DPD_DEF`, con máximos elevados que reflejan períodos prolongados de atraso, lo que convierte a estas variables en indicadores relevantes del comportamiento de pago.  
Debido a su granularidad mensual y a la relación uno-a-muchos con la tabla `previous_application`, esta tabla no puede integrarse directamente al nivel cliente y requiere agregaciones temporales previas antes de su incorporación al dataset final.


## Descripción de installments

In [ ]:
resumen_tabla(installments, "installments")

### Tabla: installments_payments

**Descripción:**  
La tabla `installments_payments` contiene el historial detallado de los pagos de cuotas correspondientes a créditos otorgados previamente por Home Credit. Permite analizar el comportamiento de pago del cliente a nivel de cada cuota, comparando montos esperados versus montos efectivamente pagados y las fechas asociadas a dichos pagos.

**Llave principal:**  
`SK_ID_PREV`

**Granularidad:**  
Cada fila representa el pago (o intento de pago) de una cuota específica asociada a una solicitud de crédito previa, identificada por el número de cuota y su versión.

**Variables relevantes:**  
`SK_ID_CURR`, `SK_ID_PREV`, `NUM_INSTALMENT_NUMBER`, `DAYS_INSTALMENT`, `DAYS_ENTRY_PAYMENT`, `AMT_INSTALMENT`, `AMT_PAYMENT`, `NUM_INSTALMENT_VERSION`

**Observaciones:**  
La tabla presenta un volumen muy elevado de registros, con más de 13,6 millones de filas y 8 columnas, todas de tipo numérico. Los valores nulos son mínimos y se concentran únicamente en `AMT_PAYMENT` y `DAYS_ENTRY_PAYMENT`, con un porcentaje inferior al 0,03%, lo que indica una alta calidad general de los datos.  
No se detectan columnas constantes ni registros duplicados. Las variables temporales `DAYS_INSTALMENT` y `DAYS_ENTRY_PAYMENT` permiten identificar atrasos o pagos anticipados al comparar la fecha programada de la cuota con la fecha real de pago.  
Se observan rangos amplios y valores extremos tanto en montos de cuotas como en montos efectivamente pagados, lo que sugiere la existencia de pagos parciales, pagos acumulados o ajustes financieros. Debido a su granularidad a nivel de cuota y a su relación uno-a-muchos con la tabla `previous_application`, esta tabla requiere agregaciones previas antes de su integración al nivel cliente.


## Descripción de credit_card

In [ ]:
resumen_tabla(credit_card, "credit_card")

### Tabla: credit_card_balance

**Descripción:**  
La tabla `credit_card_balance` contiene información mensual sobre el uso y comportamiento de las tarjetas de crédito asociadas a solicitudes de crédito previas dentro de Home Credit. Permite analizar cómo los clientes utilizan el crédito rotativo, incluyendo saldos, límites, montos girados y pagos realizados a lo largo del tiempo.

**Llave principal:**  
`SK_ID_PREV`

**Granularidad:**  
Cada fila representa el estado mensual de una tarjeta de crédito asociada a una solicitud previa, identificado por la variable `MONTHS_BALANCE`.

**Variables relevantes:**  
`SK_ID_CURR`, `SK_ID_PREV`, `MONTHS_BALANCE`, `AMT_BALANCE`, `AMT_CREDIT_LIMIT_ACTUAL`, `AMT_DRAWINGS_CURRENT`, `AMT_PAYMENT_CURRENT`, `AMT_INST_MIN_REGULARITY`

**Observaciones:**  
La tabla presenta un volumen considerable de datos, con aproximadamente 3,8 millones de registros y 23 columnas, predominando las variables numéricas. Se observan valores nulos relevantes en variables asociadas a montos y conteos de giros y pagos (`AMT_PAYMENT_CURRENT`, `CNT_DRAWINGS_*`, `AMT_DRAWINGS_*`), con porcentajes cercanos al 20%, lo que indica que no todas las tarjetas presentan actividad mensual registrada.  
No se detectan columnas constantes ni registros duplicados. Se observan valores extremos y, en algunos casos, valores negativos en variables de montos, como `AMT_BALANCE` y `AMT_DRAWINGS_*`, lo que sugiere ajustes financieros, reversos o correcciones contables.  
La variable `MONTHS_BALANCE` confirma la naturaleza temporal de la tabla, con valores negativos que representan meses en el pasado. Debido a su granularidad mensual y a la relación uno-a-muchos con la tabla `previous_application`, esta tabla requiere agregaciones previas antes de su integración al nivel cliente para su uso en el modelado.


## Descripción de columns_desc

In [ ]:
resumen_tabla(columns_desc, "columns_desc")

### Tabla: HomeCredit_columns_description

**Descripción:**  
La tabla `HomeCredit_columns_description` corresponde al diccionario de datos del dataset Home Credit. Contiene información descriptiva sobre las variables presentes en las distintas tablas del proyecto, incluyendo el nombre de la tabla, el nombre de la columna y una explicación textual de su significado.

**Llave principal:**  
No aplica (tabla de metadata).

**Granularidad:**  
Cada fila representa la descripción de una variable específica perteneciente a una tabla del dataset.

**Variables relevantes:**  
`Table`, `Row`, `Description`, `Special`

**Observaciones:**  
La tabla presenta un tamaño reducido, con 219 filas y 5 columnas, y está compuesta mayoritariamente por variables categóricas. No se identifican registros duplicados ni columnas constantes.  
Se observa un alto porcentaje de valores nulos en la columna `Special` (aproximadamente 61%), lo que indica que esta información adicional solo está disponible para un subconjunto de variables y no afecta la utilidad principal de la tabla.  
Esta tabla no se utiliza directamente en el modelado ni en la integración de datos, pero resulta fundamental para la correcta interpretación de las variables, la documentación del proyecto y la justificación de decisiones durante la ingeniería de características y la preparación de los datos.


In [28]:
def resumen_tabla(df, nombre_tabla, top_nulls=10):
    print("="*80)
    print(f"RESUMEN DATA UNDERSTANDING - {nombre_tabla}")
    print("="*80)

    # Dimensiones
    print("\nDimensiones (filas, columnas):")
    print(df.shape)

    # Tipos de datos
    print("\nTipos de datos:")
    print(df.dtypes.value_counts())

    # Columnas categóricas y numéricas
    cat_cols = df.select_dtypes(include="object").columns
    num_cols = df.select_dtypes(include=["int64", "float64"]).columns

    print("\nNúmero de columnas categóricas:", len(cat_cols))
    print("Número de columnas numéricas:", len(num_cols))

    # Valores nulos
    nulls = df.isnull().sum()
    null_percent = (nulls / len(df)) * 100

    nulls_df = pd.DataFrame({
        "Nulos": nulls,
        "% Nulos": null_percent
    }).sort_values("% Nulos", ascending=False)

    print(f"\nTop {top_nulls} columnas con más valores nulos:")
    display(nulls_df.head(top_nulls))

    # Columnas con alto porcentaje de nulos
    high_nulls = nulls_df[nulls_df["% Nulos"] > 40]
    print("\nColumnas con más de 40% de valores nulos:")
    display(high_nulls)

    # Columnas constantes
    constant_cols = [c for c in df.columns if df[c].nunique(dropna=False) <= 1]
    print("\nColumnas constantes o con un solo valor:")
    print(constant_cols)

    # Duplicados
    duplicates = df.duplicated().sum()
    print("\nFilas duplicadas:", duplicates)

    # Estadísticas básicas (solo numéricas)
    if len(num_cols) > 0:
        print("\nEstadísticas básicas (primeras 10 columnas numéricas):")
        display(df[num_cols].describe().T.head(10))

    print("\nFin del resumen de la tabla.")